In [1]:
from pyMSOO.MFEA.model import SBSGA
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.utils.DimensionAwareStrategy import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 
from pyMSOO.utils.MultiRun.RunMultiBenchmark import * 

In [2]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(1)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()

In [3]:
baseModel = SBSGA.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy=DaS_strategy(eta= 3)
)
solve = baseModel.fit(
    nb_generations = 1000, rmp = 0.5, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Exception ignored on calling ctypes callback function: <function ExecutionEngine._raw_object_cache_notify at 0x00000237AED459D0>
Traceback (most recent call last):
  File "f:\Anaconda\Anaconda\envs\MFEA\lib\site-packages\llvmlite\binding\executionengine.py", line 171, in _raw_object_cache_notify
    def _raw_object_cache_notify(self, data):
KeyboardInterrupt: 


### MUltitime Model

In [3]:
baseModel = MultiBenchmark(model= SBSGA, ls_benchmark=[tasks], ls_IndClass=[IndClass], name_benchmark = ['CEC17'])
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy= NoDaS(),
)
solve = baseModel.fit(
    nb_generations = 1000,nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

In [5]:
baseModel.run(
    nb_run= 2,
    save_path= './RESULTS/SBSGA'
)

Seed: 0 -- Time: 01m 37.31s  100 % [====================>]  Cost: 1.04E-06  3.25E-07  1.60E-08  0.00E+00  1.45E-04  1.63E-04  7.47E-03  6.38E-04  1.84E-08  2.28E+01  ,  
END!
Seed: 1 -- Time: 01m 29.96s  100 % [====================>]  Cost: 2.42E-06  3.30E-07  1.44E-10  0.00E+00  3.28E-04  1.77E-04  1.63E-03  6.39E-04  2.45E-08  2.65E+01  ,  
END!
DONE!
Saved
